In [ ]:
!pip install "lxml[html_clean]"

In [ ]:
!pip install langchain langchain-community langchain-groq langchain-experimental google-search-results pydantic newspaper3k beautifulsoup4 chromadb duckduckgo-search tiktoken

In [ ]:
!pip install sentence-transformers

In [40]:
import os
import requests
from newspaper import Article
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS

from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.utilities import SerpAPIWrapper
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_community.embeddings import HuggingFaceEmbeddings

import getpass

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your GROQ API key: ")
os.environ["SERPAPI_API_KEY"] = getpass.getpass("Enter your SERPAPI API key: ")


llm = ChatGroq(api_key=os.environ["GROQ_API_KEY"], model_name="llama3-70b-8192")



<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()


Enter your GROQ API key: ··········
Enter your SERPAPI API key: ··········


In [41]:
def duckduckgo_search(query, max_results=5):
    with DDGS() as ddgs:
        results = [r for r in ddgs.text(query, max_results=max_results)]
        return [r["href"] for r in results if "href" in r]



In [42]:
def extract_article(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception:
        try:
            html = requests.get(url, timeout=5).text
            soup = BeautifulSoup(html, "html.parser")
            return " ".join([p.get_text() for p in soup.find_all("p")])
        except:
            return ""



In [43]:
def embed_to_vectorstore(texts):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain_community.vectorstores import Chroma

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.create_documents(texts)

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    db = Chroma.from_documents(chunks, embedding=embeddings, persist_directory="./chroma")
    db.persist()
    return db



In [44]:
class ResearchSummary(BaseModel):
    title: str = Field(description="Title of the research topic")
    summary: str = Field(description="Short summary in 2-3 lines")
    citations: list[str] = Field(description="List of URLs used for reference")

parser = PydanticOutputParser(pydantic_object=ResearchSummary)

summary_prompt = PromptTemplate(
    template="Summarize this research topic: {topic}\n\nBased on the following sources:\n{sources}\n\n{format_instructions}",
    input_variables=["topic", "sources"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

def structured_summary(topic: str, urls: list, texts: list) -> str:
    input_text = "\n\n".join(texts[:3])  # Use first 3 articles
    prompt = summary_prompt.format(topic=topic, sources="\n".join(urls))
    response = llm.invoke(prompt)
    parsed = parser.parse(response.content)
    return f"""📘 **{parsed.title}**

📝 **Summary:** {parsed.summary}

🔗 **Citations:**
""" + "\n".join(f"- {url}" for url in parsed.citations)



In [45]:
def web_research_tool(query: str) -> str:
    urls = duckduckgo_search(query, max_results=5)
    articles = [extract_article(url) for url in urls]
    articles = [t for t in articles if len(t.strip()) > 100]  # filter short/noisy
    embed_to_vectorstore(articles)
    return structured_summary(query, urls, articles)

tools = [
    Tool(
        name="Research Summary Generator",
        func=web_research_tool,
        description="Fetches and summarizes recent research from the web"
    )
]



In [46]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)



In [47]:
if __name__ == "__main__":
    query = "Compare LangChain vs CrewAI for AI agent development"
    print("🔍 Query:", query)
    result = agent.invoke(query)
    print("\n✅ Final Result:\n", result)


🔍 Query: Compare LangChain vs CrewAI for AI agent development


> Entering new AgentExecutor chain...
Thought: I need to research and compare LangChain and CrewAI for AI agent development.

Action: Research Summary Generator
Action Input: "LangChain vs CrewAI for AI agent development"


Streaming output truncated to the last 5000 lines.
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; fal

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...


Observation: 📘 **LangChain vs CrewAI for AI Agent Development**

📝 **Summary:** This research topic compares two AI agent development frameworks, LangChain and CrewAI, highlighting their features, advantages, and use cases. The comparison aims to help developers choose the best framework for their AI agent development needs.

🔗 **Citations:**
- https://www.analyticsvidhya.com/blog/2025/03/agent-sdk-vs-crewai-vs-langchain/
- https://iartificial.blog/en/automation/Crewai-vs.-Langchain:-Best-AI-Agent-Framework-2025/
- https://medium.com/@harshachaitanya27/comparing-ai-agent-platforms-crewai-autogen-langchain-and-pydantic-ai-163a01b77136
- https://www.linkedin.com/pulse/langchain-vs-crewai-microsoft-autogen-comparison-deepak-devassykutty-8awoc
- https://www.gocodeo.com/post/choosing-the-right-ai-agent-framework-langchain-vs-crewai-vs-autogen
Thought:Thought: Now that I have the research summary, I need to analyze the key points and create a comparison between LangChain and CrewAI.

Action

/tmp/ipython-input-41-2009479495.py:4: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/usr/local/lib/python3.11/dist-packages/requests/utils.py:494: DeprecationWarning: In requests 3.0, get_encodings_from_content will be removed. For more information, please see the discussion on issue #2266. (This warning should only appear once.)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/requests/utils.py:494: DeprecationWarning: In requests 3.0, get_encodings_from_content will be removed. For more information, please see the discussion on issue #2266. (This warning should only appear once.)
  warnings.warn(
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()



Observation: 📘 **Key features and advantages of LangChain and CrewAI for AI agent development**

📝 **Summary:** LangChain and CrewAI are two popular AI agent development frameworks that offer distinct advantages and features. LangChain is known for its flexibility and customizability, while CrewAI excels in ease of use and rapid development. The choice between the two ultimately depends on the specific needs and goals of the project.

🔗 **Citations:**
- https://iartificial.blog/en/automation/Crewai-vs.-Langchain:-Best-AI-Agent-Framework-2025/
- https://medium.com/ai-simplified-in-plain-english/which-ai-tool-should-you-choose-agent-sdk-langchain-and-crewai-for-building-powerful-ai-agents-f797dc3fa79b
- https://orq.ai/blog/langchain-vs-crewai
- https://www.gocodeo.com/post/choosing-the-right-ai-agent-framework-langchain-vs-crewai-vs-autogen
- https://blog.gopenai.com/langchain-vs-crewai-benchmarking-12-key-design-traits-that-define-ai-agent-intelligence-159e12030e5c
Thought:Thought: I h

/tmp/ipython-input-41-2009479495.py:4: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()



Observation: 📘 **Comparison of LangChain and CrewAI for AI Agent Development**

📝 **Summary:** This research topic compares LangChain and CrewAI, two platforms for AI agent development. The comparison aims to highlight their differences and similarities, providing insights for developers and researchers.

🔗 **Citations:**
- https://www.merriam-webster.com/dictionary/comparison
- https://dictionary.cambridge.org/dictionary/english/comparison
- https://en.wikipedia.org/wiki/Comparison
- https://www.collinsdictionary.com/dictionary/english/comparison
- https://www.thefreedictionary.com/comparison
Thought:Thought: I now know the final answer.

Final Answer: 

LangChain and CrewAI are both AI agent development frameworks with distinct advantages and features. LangChain is known for its flexibility and customizability, while CrewAI excels in ease of use and rapid development. The choice between the two ultimately depends on the specific needs and goals of the project.

Here is a comparison 